In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using ExactDiagonalizationTools
using MatrixProductStateTools
using TensorNetAlgs
using Plots

In [ ]:
# Find the ground sate Ising Transverse Field model
lx = 100
tol = 1.e-9
maxdim = 200

mps = MatrixProductState{Float64}(lx, 2, noise=0.2);

mpo = qitf_mpo(Float64, lx, 2, 0.9)
env = initialenv(mps, mpo)

for sweep = 1:4
    println("Starting sweep number $sweep")
    dmrg2sitesweep!(mps, mpo, env, maxdim, tol, verbose=false)
    #println()
end

In [ ]:
sz = Matrix(sz_half)
data = []
sx = Matrix(sp_half + sm_half)/2.
half = div(lx, 2)
for l = half-2:half+2
    apply_1siteoperator!(mps, l, 2 .*sx)
end
push!(data, measure_1point(mps, sz))
dt = 0.01
mpsc = convert(MatrixProductState{ComplexF64}, mps)
mpoc = convert(MatrixProductOperator{ComplexF64}, mpo)
env = initialenv(mpsc, mpoc)
for t=1:20
    for i = 1:50
        tdvp1sitesweep!(dt, mpsc, mpoc, env, verbose=false)
    end
    push!(data, real.(measure_1point(mpsc, sz)))
end

In [ ]:
p = plot()
for i=1:length(data)
    plot!(p, 1:lx, data[i])
end
p